In [ ]:
import pandas as pd
from functions import load_data, clean_data, style_table

def main():
    # Load and clean data
    url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"
    df_sa = load_data(url)
    df_sa = clean_data(df_sa)

    # Ask the user if they want to display the table
    show_table = input("Show table? Y/N: ").strip().lower()

    if show_table == 'y':
        styled_df = style_table(df_sa)
        display(styled_df)
    
    display(df_sa.head(10))

if __name__ == "__main__":
    main()

In [49]:
import pandas as pd

url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"

df_sa = pd.read_excel(url)

df_sa.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,15 Mar 2024,2024.0,Unprovoked,AUSTRALIA,Queensland,Bargara Beach,Swimming,Brooklyn Sauer,F,13,...,Tiger shark,"Yahoo News, 3/15/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04 Mar 2024,2024.0,Unprovoked,USA,Hawaii,"Old Man's, Waikiki",Surfing,Matthew White,M,NaN,...,Tiger shark 8',"Surfer, 3/6/2024F",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02 Mar-2024,2024.0,Unprovoked,USA,Hawaii,"Rainbows, Oahu",Swimming,NaN,F,11,...,3' to 4' shark,"Hawaii News Now, 3/4/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25 Feb-2024,2024.0,Unprovoked,AUSTRALIA,Western Australia,"Sandlnd Island, Jurian Bay",NaN,female,F,46,...,Tiger shark,"WA Today, 2/26/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14 Feb-2024,2024.0,Unprovoked,INDIA,Maharashtra,"Vaitarna River, Palghar District",Fishing,Vicky Suresh Govari,M,32,...,"Bull shark, 7'","Times of India, 2/14/2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_sa["Year"] 

0       2024.0
1       2024.0
2       2024.0
3       2024.0
4       2024.0
         ...  
6964       NaN
6965       NaN
6966       NaN
6967       NaN
6968       NaN
Name: Year, Length: 6969, dtype: float64

In [30]:
# YEAR

df_sa["Year"] = df_sa["Year"].fillna(0).astype(int)             # I change NA for 0 in order to avoid errors when parsing float to int

df_sa["Year"].value_counts()

Year
0       156
2015    143
2017    139
2016    131
2011    128
       ... 
1785      1
1784      1
1783      1
1780      1
5         1
Name: count, Length: 259, dtype: int64

In [50]:
df_sa["Type"].value_counts()

Type
Unprovoked             5107
Provoked                635
Invalid                 552
Watercraft              355
Sea Disaster            242
Questionable             22
Boat                      7
 Provoked                 2
?                         1
Unconfirmed               1
Unverified                1
Under investigation       1
Name: count, dtype: int64

In [60]:
# TYPE

for x in df_sa["Type"]:
    if x == str:
        
        df_sa["Type"] = df_sa["Type"].str.replace(x, "xD")
    else: 
        print(x)

df_sa["Type"].value_counts()

Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
 Provoked
Unprovoked
Unprovoked
Unprovoked
Provoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Questionable
Questionable
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Questionable
Unprovoked
Unprovoked
Provoked
Watercraft
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Questionable
Unprovoked
Unprovoked
Watercraft
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Watercraft
Questionable
Unprovoked
Provoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Watercraft
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Questionable
Unprovoked
Provoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked
Provoked
 Provoked
Unprovoked
Unprovoked
Unprovoked
Unprovoked

Type
Unprovoked             5107
Provoked                635
Invalid                 552
Watercraft              355
Sea Disaster            242
Questionable             22
Boat                      7
 Provoked                 2
?                         1
Unconfirmed               1
Unverified                1
Under investigation       1
Name: count, dtype: int64